### КР2

### Бустинг. - 6 баллов
В существующий код бустинга добавьте возможность ранней остановки обучения. 
должны быть учтены:
1) Наличие валидационного датасета (либо разделение должно быть внутри класса, либо вне его, а в обучении новый набор будет подаваться отдельной парой)
2) Кастомная метрика или лосс для оствновки. Должна передаваться в виде доп. параметра. Дефолт - лосс функция для расчета градиента.
3) Укажите, сколько должно пройти итераций для ранней остановки. 
4) После обучения должно вернуться лучшее состояние модели по валидационной выборке, а не то, которое было достинуто при остановке обучения. 

Для обучения используйте тот же датасет, что использовался на 8 семинаре (house_price_regression_dataset).
1 и 3 пункты обязательны - 3 балла. 2 пункт - 1 балл (при недефолтной реализации). 4 пункт - 2 балла.

In [27]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor


### Собственная реализация
class MyGradientRegressor:
    def __init__(self, n_estimators: int = 300, max_depth: int = 3, lr: float = 0.1):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.lr = lr
        self.estimators = []

    def fit(self, X_train, y_train):
        X_train = np.array(X_train)
        y_train = np.array(y_train)

        self.estimators = []
        predictions = 0

        for _ in range(self.n_estimators):
            new_model = DecisionTreeRegressor(max_depth=self.max_depth)
            new_target = -2 * (predictions - y_train)
            new_model.fit(X_train, new_target)
            predictions += self.lr * new_model.predict(X_train)
            self.estimators.append(new_model)

    def predict(self, X_test):
        X_test = np.array(X_test)
        curr_pred = 0
        for est in self.estimators:
            curr_pred += self.lr * est.predict(X_test)

        return curr_pred

## Решение задания "Бустинг"

In [28]:
from sklearn.metrics import mean_squared_error

class MyGradientRegressor:
    def __init__(self, n_estimators=300, max_depth=3, early_stop_rounds=10, eval_metric=None):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.early_stop_rounds = early_stop_rounds
        self.eval_metric = eval_metric if eval_metric else mean_squared_error
        self.estimators = []
        self.best_estimators = []

    def fit(self, X_train, y_train, X_valid, y_valid):
        self.estimators = []
        best_score = np.inf
        no_improve_count = 0
        preds = np.zeros(X_train.shape[0])

        for i in range(self.n_estimators):
            model = DecisionTreeRegressor(max_depth=self.max_depth)
            gradient = y_train - preds
            model.fit(X_train, gradient)

            preds += model.predict(X_train)
            self.estimators.append(model)

            # Проверка
            pred_valid = self.predict(X_valid)
            valid_score = self.eval_metric(y_valid, pred_valid)

            if valid_score < best_score:
                best_score = valid_score
                no_improve_count = 0
                self.best_estimators = self.estimators.copy()
            else:
                no_improve_count += 1

            if no_improve_count >= self.early_stop_rounds:
                print(f"Ранняя остановка после {i+1} итераций")
                break

        self.estimators = self.best_estimators.copy()

    def predict(self, X):
        preds_sum = np.zeros(X.shape[0])
        for tree in self.estimators:
            preds_sum += tree.predict(X)
        return preds_sum


### Проверка

In [29]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

dom_df = pd.read_csv('house_price_regression_dataset.csv')

X_dom = dom_df.drop('House_Price', axis=1)
y_cena = dom_df['House_Price']
X_train, X_test, y_train, y_test = train_test_split(X_dom, y_cena, test_size=0.2, random_state=42)

# Нормировка
norma = StandardScaler()
X_train_norm = norma.fit_transform(X_train)
X_test_norm = norma.transform(X_test)

# Валидационный сплит (для ранней остановки)
X_train_osn, X_valid, y_train_osn, y_valid = train_test_split(X_train_norm, y_train, test_size=0.2, random_state=42)

# Обучение модели с ранней остановкой
boost_modelka = MyGradientRegressor(n_estimators=100, max_depth=3, early_stop_rounds=5)
boost_modelka.fit(X_train_osn, y_train_osn, X_valid, y_valid)

# Проверка
pred_valid = boost_modelka.predict(X_valid)
mse_valid = mean_squared_error(y_valid, pred_valid)
print(f"MSE на валидации после ранней остановки: {mse_valid:.1f}")

pred_test = boost_modelka.predict(X_test_norm)
mse_test = mean_squared_error(y_test, pred_test)
print(f"MSE на тестовом наборе: {mse_test:.1f}")



Ранняя остановка после 33 итераций
MSE на валидации после ранней остановки: 402124171.7
MSE на тестовом наборе: 509017399.8


### Стекинг - 4 балла
В текущей реализации в качестве признаков для метамодели используются предсказания базовых моделей.
Ваша задача добавить возможность дополнительно учитывать исходные данные в качестве признаков (гиперпараметр). 
Метапризнаки как доп. фичи к основным.
При этом на основные признаки добавляется воможность расчета полиномиальных признаков (https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html)

Для тестирования используйте тот же датасет

In [30]:
from sklearn.model_selection import KFold


class Stacking:
    def __init__(self, estimators, meta_estimator, folds=5):
        self.estimators = estimators
        self.meta_estimator = meta_estimator
        self.folds = folds
        self.meta_train = []

    def _fit_estimator(self, estimator, X_train, y_train):
        kf = KFold(n_splits=self.folds, shuffle=True)
        train_fold_indices = []
        test_fold_indices = []
        test_fold_predicts = []

        for train_idx, test_idx in kf.split(X_train):
            train_fold_indices.extend(train_idx)
            test_fold_indices.extend(test_idx)

            estimator.fit(X_train[train_idx], y_train[train_idx])
            test_fold_predicts.extend(estimator.predict(X_train[test_idx]))

        estimator.fit(X_train, y_train)
        self.meta_train.append(np.array(test_fold_predicts)[np.argsort(test_fold_indices)])

    def fit(self, X_train, y_train):
        X_train = np.array(X_train)
        y_train = np.array(y_train)
        self.meta_train = []

        for estimator in self.estimators:
            self._fit_estimator(estimator, X_train, y_train)

        self.meta_train = np.array(self.meta_train).transpose()
        self.meta_estimator.fit(self.meta_train, y_train)

    def predict(self, X_test):
        X_test = np.array(X_test)
        meta_features = np.array([estimator.predict(X_test) for estimator in self.estimators]).transpose()
        return self.meta_estimator.predict(meta_features)

## Решение

In [31]:
from sklearn.preprocessing import PolynomialFeatures

class Stacking:
    def __init__(self, estimators, meta_estimator, folds=5, use_original_features=False, poly_degree=2):
        self.estimators = estimators
        self.meta_estimator = meta_estimator
        self.folds = folds
        self.use_original_features = use_original_features
        self.poly_degree = poly_degree
        self.poly = None

    def _fit_estimator(self, estimator, X, y):
        kf = KFold(n_splits=self.folds, shuffle=True, random_state=42)
        preds = np.zeros(X.shape[0])

        for train_idx, test_idx in kf.split(X):
            estimator.fit(X[train_idx], y[train_idx])
            preds[test_idx] = estimator.predict(X[test_idx])

        estimator.fit(X, y)
        return preds

    def fit(self, X_train, y_train):
        meta_features = []
        for estimator in self.estimators:
            preds = self._fit_estimator(estimator, X_train, y_train)
            meta_features.append(preds)

        meta_features = np.array(meta_features).T

        if self.use_original_features:
            self.poly = PolynomialFeatures(degree=self.poly_degree, include_bias=False)
            original_poly = self.poly.fit_transform(X_train)
            meta_features = np.hstack([meta_features, original_poly])

        self.meta_estimator.fit(meta_features, y_train)

    def predict(self, X_test):
        meta_features = []
        for estimator in self.estimators:
            preds = estimator.predict(X_test)
            meta_features.append(preds)

        meta_features = np.array(meta_features).T

        if self.use_original_features and self.poly is not None:
            original_poly = self.poly.transform(X_test)
            meta_features = np.hstack([meta_features, original_poly])

        return self.meta_estimator.predict(meta_features)


## Проверка

In [33]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression

dom_df = pd.read_csv('house_price_regression_dataset.csv')
X = dom_df.drop('House_Price', axis=1).values  # класс ест numpy array
y = dom_df['House_Price'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

base_estimators = [
    DecisionTreeRegressor(max_depth=4, random_state=42),
    DecisionTreeRegressor(max_depth=5, random_state=42)
]

meta_estimator = LinearRegression()

stacking_model = Stacking(
    estimators=base_estimators,
    meta_estimator=meta_estimator,
    folds=5,
    use_original_features=True,
    poly_degree=2
)

# Обучение
stacking_model.fit(X_train_scaled, y_train)

# Предсказание и оценка
y_pred = stacking_model.predict(X_test_scaled)
mse_test = mean_squared_error(y_test, y_pred)

print(f"MSE на тестовом наборе: {mse_test:.1f}")


MSE на тестовом наборе: 103526070.2
